In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import sys
# !{sys.executable} -m pip install torch

In [ ]:
import pandas as pd
import numpy as np

import torch 
import torch.nn.functional as fn

from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt

Step 1: Generate Triplet Set from iBKH

In [ ]:
#kg_folder = '/content/drive/MyDrive/DSW_Project/Dataset/' # The folder is used to store the iBKH results, include Entity results and Relation results
kg_folder = 'Entity/'

In [ ]:
# Extracting triplets between drug and disease entities

def DDi_triplets(): 
    ddi = pd.read_csv(kg_folder + 'Relation/D_Di_res.csv')

    ddi_treats = ddi[ddi['Treats'] == 1]
    ddi_treats['Relation'] = ['Treats_DDi'] * len(ddi_treats)
    ddi_treats = ddi_treats[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_palliates = ddi[ddi['Palliates'] == 1]
    ddi_palliates['Relation'] = ['Palliates_DDi'] * len(ddi_palliates)
    ddi_palliates = ddi_palliates[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_effect = ddi[ddi['Effect'] == 1]
    ddi_effect['Relation'] = ['Effect_DDi'] * len(ddi_effect)
    ddi_effect = ddi_effect[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_associate = ddi[ddi['Associate'] == 1]
    ddi_associate['Relation'] = ['Associate_DDi'] * len(ddi_associate)
    ddi_associate = ddi_associate[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_IR = ddi[ddi['Inferred_Relation'] == 1]
    ddi_IR['Relation'] = ['Inferred_Relation_DDi'] * len(ddi_IR)
    ddi_IR = ddi_IR[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_SR = ddi[
        (ddi['treatment/therapy (including investigatory)'] == 1) | (ddi['inhibits cell growth (esp. cancers)'] == 1) |
        (ddi['alleviates, reduces'] == 1) | (ddi['biomarkers (of disease progression)'] == 1) |
        (ddi['prevents, suppresses'] == 1) | (ddi['role in disease pathogenesis'] == 1)]
    ddi_SR['Relation'] = ['Semantic_Relation_DDi'] * len(ddi_SR)
    ddi_SR = ddi_SR[['Drug', 'Relation', 'Disease', 'Inference_Score']]

    ddi_res = pd.concat((ddi_treats, ddi_palliates, ddi_effect, ddi_associate, ddi_IR, ddi_SR))
    ddi_res = ddi_res.rename(columns={'Drug': 'Head', 'Disease': 'Tail'})

    ddi_res.loc[ddi_res['Relation'] != 'Inferred_Relation_DDi', 'Inference_Score'] = np.nan

    ddi_res.to_csv('DDi_triplet.csv', index=False)

In [ ]:
# Extracting triplets between drug and gene entities.

def DG_triplets():
    dg = pd.read_csv(kg_folder + 'Relation/D_G_res.csv')

    dg_target = dg[dg['Target'] == 1]
    dg_target['Relation'] = ['Target_DG'] * len(dg_target)
    dg_target['Inference_Score'] = [''] * len(dg_target)
    dg_target = dg_target[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_transporter = dg[dg['Transporter'] == 1]
    dg_transporter['Relation'] = ['Transporter_DG'] * len(dg_transporter)
    dg_transporter['Inference_Score'] = [''] * len(dg_transporter)
    dg_transporter = dg_transporter[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_enzyme = dg[dg['Enzyme'] == 1]
    dg_enzyme['Relation'] = ['Enzyme_DG'] * len(dg_enzyme)
    dg_enzyme['Inference_Score'] = [''] * len(dg_enzyme)
    dg_enzyme = dg_enzyme[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_carrier = dg[dg['Carrier'] == 1]
    dg_carrier['Relation'] = ['Carrier_DG'] * len(dg_carrier)
    dg_carrier['Inference_Score'] = [''] * len(dg_carrier)
    dg_carrier = dg_carrier[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_downregulates = dg[dg['Downregulates'] == 1]
    dg_downregulates['Relation'] = ['Downregulates_DG'] * len(dg_downregulates)
    dg_downregulates['Inference_Score'] = [''] * len(dg_downregulates)
    dg_downregulates = dg_downregulates[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_upregulates = dg[dg['Upregulates'] == 1]
    dg_upregulates['Relation'] = ['Upregulates_DG'] * len(dg_upregulates)
    dg_upregulates['Inference_Score'] = [''] * len(dg_upregulates)
    dg_upregulates = dg_downregulates[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_associate = dg[dg['Associate'] == 1]
    dg_associate['Relation'] = ['Associate_DG'] * len(dg_associate)
    dg_associate['Inference_Score'] = [''] * len(dg_associate)
    dg_associate = dg_associate[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_binds = dg[dg['Binds'] == 1]
    dg_binds['Relation'] = ['Binds_DG'] * len(dg_binds)
    dg_binds['Inference_Score'] = [''] * len(dg_binds)
    dg_binds = dg_binds[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_interaction = dg[dg['Interaction'] == 1]
    dg_interaction['Relation'] = ['Interaction_DG'] * len(dg_interaction)
    dg_interaction['Inference_Score'] = [''] * len(dg_interaction)
    dg_interaction = dg_interaction[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_SR = dg[(dg['affects expression/production (neutral)'] == 1) | (dg['agonism, activation'] == 1) |
               (dg['inhibits'] == 1) | (dg['metabolism, pharmacokinetics'] == 1) | (dg['antagonism, blocking'] == 1) |
               (dg['increases expression/production'] == 1) | (dg['binding, ligand (esp. receptors)'] == 1) |
               (dg['decreases expression/production'] == 1) | (dg['transport, channels'] == 1) |
               (dg['enzyme activity'] == 1) | (dg['physical association'] == 1)]
    dg_SR['Relation'] = ['Semantic_Relation_DG'] * len(dg_SR)
    dg_SR['Inference_Score'] = [''] * len(dg_SR)
    dg_SR = dg_SR[['Drug', 'Relation', 'Gene', 'Inference_Score']]

    dg_res = pd.concat(
        (dg_target, dg_transporter, dg_enzyme, dg_carrier, dg_downregulates, dg_upregulates, dg_associate,
         dg_binds, dg_interaction, dg_SR))
    dg_res = dg_res.rename(columns={'Drug': 'Head', 'Gene': 'Tail'})

    dg_res.to_csv('DG_triplet.csv', index=False)

In [ ]:
# Extracting triplets between disease and gene entities.

def DiG_triplets():
    dig = pd.read_csv(kg_folder + 'Relation/Di_G_res.csv')

    dig_associate = dig[dig['Associate'] == 1]
    dig_associate['Relation'] = ['Associate_DiG'] * len(dig_associate)
    dig_associate = dig_associate[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_downregulates = dig[dig['Downregulates'] == 1]
    dig_downregulates['Relation'] = ['Downregulates_DiG'] * len(dig_downregulates)
    dig_downregulates = dig_downregulates[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_upregulates = dig[dig['Upregulates'] == 1]
    dig_upregulates['Relation'] = ['Upregulates_DiG'] * len(dig_upregulates)
    dig_upregulates = dig_upregulates[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_IR = dig[dig['Inferred_Relation'] == 1]
    dig_IR['Relation'] = ['Inferred_Relation_DiG'] * len(dig_IR)
    dig_IR = dig_IR[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_SR = dig[(dig['improper regulation linked to disease'] == 1) | (dig['causal mutations'] == 1) |
                 (dig['polymorphisms alter risk'] == 1) | (dig['role in pathogenesis'] == 1) |
                 (dig['possible therapeutic effect'] == 1) | (dig['biomarkers (diagnostic)'] == 1) |
                 (dig['promotes progression'] == 1) | (dig['drug targets'] == 1) | (
                         dig['overexpression in disease'] == 1) |
                 (dig['mutations affecting disease course'] == 1)]
    dig_SR['Relation'] = ['Semantic_Relation_DiG'] * len(dig_SR)
    dig_SR = dig_SR[['Disease', 'Relation', 'Gene', 'Inference_Score']]

    dig_res = pd.concat((dig_associate, dig_downregulates, dig_upregulates, dig_IR, dig_SR))
    dig_res = dig_res.rename(columns={'Disease': 'Head', 'Gene': 'Tail'})

    dig_res.loc[dig_res['Relation'] != 'Inferred_Relation_DiG', 'Inference_Score'] = np.nan

    dig_res.to_csv('DiG_triplet.csv', index=False)

In [ ]:
# Extracting triplets between gene entities.

def GG_triplets():
    gg = pd.read_csv(kg_folder + 'Relation/G_G_res.csv')

    gg_covaries = gg[gg['Covaries'] == 1]
    gg_covaries['Relation'] = ['Covaries_GG'] * len(gg_covaries)
    gg_covaries['Inference_Score'] = [''] * len(gg_covaries)
    gg_covaries = gg_covaries[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_interacts = gg[gg['Interacts'] == 1]
    gg_interacts['Relation'] = ['Interacts_GG'] * len(gg_interacts)
    gg_interacts['Inference_Score'] = [''] * len(gg_interacts)
    gg_interacts = gg_interacts[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_regulates = gg[gg['Regulates'] == 1]
    gg_regulates['Relation'] = ['Regulates_GG'] * len(gg_regulates)
    gg_regulates['Inference_Score'] = [''] * len(gg_regulates)
    gg_regulates = gg_regulates[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_associate = gg[gg['Associate'] == 1]
    gg_associate['Relation'] = ['Associate_GG'] * len(gg_associate)
    gg_associate['Inference_Score'] = [''] * len(gg_associate)
    gg_associate = gg_associate[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_SR = gg[
        (gg['activates, stimulates'] == 1) | (gg['production by cell population'] == 1) | (gg['regulation'] == 1) |
        (gg['binding, ligand (esp. receptors)'] == 1) | (gg['signaling pathway'] == 1) |
        (gg['increases expression/production'] == 1) | (gg['same protein or complex'] == 1) |
        (gg['enhances response'] == 1) | (gg['affects expression/production (neutral)'] == 1) |
        (gg['physical association'] == 1) | (gg['association'] == 1) | (gg['colocalization'] == 1) |
        (gg['dephosphorylation reaction'] == 1) | (gg['cleavage reaction'] == 1) | (gg['direct interation'] == 1) |
        (gg['ADP ribosylation reaction'] == 1) | (gg['ubiquitination reaction'] == 1) |
        (gg['phosphorylation reaction'] == 1) | (gg['protein cleavage'] == 1)]
    gg_SR['Relation'] = ['Semantic_Relation_GG'] * len(gg_SR)
    gg_SR['Inference_Score'] = [''] * len(gg_SR)
    gg_SR = gg_SR[['Gene_1', 'Relation', 'Gene_2', 'Inference_Score']]

    gg_res = pd.concat((gg_covaries, gg_interacts, gg_regulates, gg_associate, gg_SR))
    gg_res = gg_res.rename(columns={'Gene_1': 'Head', 'Gene_2': 'Tail'})

    gg_res.to_csv('GG_triplet.csv', index=False)

In [ ]:
DDi_triplets()
DG_triplets()
DiG_triplets()
GG_triplets()

In [ ]:
# This function will return a csv file, which combines all triplets for interest with removing 
# AD-related drug-disease triplets

# def generate_training_set():
#     disease_vocab = pd.read_csv(kg_folder + 'disease_vocab.csv')
#     AD_related_list = [] # Find AD related entities from the Disease vocabulary in iBKH
#     for i in range(len(disease_vocab)):
#         primary_id = disease_vocab.loc[i, 'primary']
#         disease_name = disease_vocab.loc[i, 'name']
#         disease_name = disease_name if not pd.isnull(disease_name) else ''
#         if 'alzheimer' in disease_name:
#             if primary_id not in AD_related_list:
#                 AD_related_list.append(primary_id)

#     triplet_folder = ''
#     DDi_triplet = pd.read_csv(triplet_folder + 'DDi_triplet.csv')
#     DDi_triplet = DDi_triplet[~(DDi_triplet['Tail'].isin(AD_related_list))]
#     DDi_triplet = DDi_triplet.reset_index(drop=True)
#     DG_triplet = pd.read_csv(triplet_folder + 'DG_triplet.csv')
#     DiG_triplet = pd.read_csv(triplet_folder + 'DiG_triplet.csv')
#     GG_triplet = pd.read_csv(triplet_folder + 'GG_triplet.csv')
#     DD_triplet = pd.read_csv(triplet_folder + 'Triplets/DD_triplet.csv')

#     triplet_set = pd.concat((DDi_triplet, DG_triplet, DiG_triplet, GG_triplet, DD_triplet))
#     triplet_set = triplet_set[['Head', 'Relation', 'Tail']]

#     triplet_set.to_csv('training_triplets.csv', index=False)

# generate_training_set()  

In [ ]:
# This function will return a csv file, which combines all triplets for interest ###WITHOUT### removing 
# AD-related drug-disease triplets

def generate_training_set_wAD():
    disease_vocab = pd.read_csv(kg_folder + 'disease_vocab.csv')
    

    triplet_folder = ''
    DDi_triplet = pd.read_csv(triplet_folder + 'DDi_triplet.csv')
    DG_triplet = pd.read_csv(triplet_folder + 'DG_triplet.csv')
    DiG_triplet = pd.read_csv(triplet_folder + 'DiG_triplet.csv')
    GG_triplet = pd.read_csv(triplet_folder + 'GG_triplet.csv')
    DD_triplet = pd.read_csv(triplet_folder + 'Triplets/DD_triplet.csv')

    triplet_set = pd.concat((DDi_triplet, DG_triplet, DiG_triplet, GG_triplet, DD_triplet))
    triplet_set = triplet_set[['Head', 'Relation', 'Tail']]

    triplet_set.to_csv('wAD_training_triplets.csv', index=False)

generate_training_set_wAD()  

In [ ]:
import os
def generate_training_set_wAd():
    triplets_set = pd.read_csv('wAd_training_triplets.csv')

    triples = triplets_set.values.tolist()
    train_set = np.arange(len(triples)).tolist()
    
    dataset_path = 'FinalProject'
    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)

    print(len(triples), len(train_set))
    with open(dataset_path + "wAD_training_triplets.tsv", 'w+') as f:
        for idx in train_set:
            f.writelines("{}\t{}\t{}\n".format(triples[idx][0], triples[idx][1], triples[idx][2]))
            
generate_training_set_wAd()


In [3]:
torch.cuda.is_available()


False

In [ ]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # set the default GPU as device 0
else:
    device = torch.device("cpu")

# Set default device as GPU
torch.cuda.set_device(device)

In [ ]:
DGLBACKEND=pytorch dglke_train --dataset iBKH --data_path ./FinalProject --data_files wAD_training_triplets.tsv --format raw_udd_hrt --model_name TransE_l2 --batch_size 1024 --neg_sample_size 256 --hidden_dim 200 --gamma 12.0 --lr 0.1 --max_step 10000 --log_interval 100 --batch_size_eval 1000 -adv --regularization_coef 1.00E-07 --num_thread 1 --num_proc 8 --neg_sample_size_eval 1000

In [ ]:
DGLBACKEND=pytorch dglke_train --dataset iBKH --data_path ./dataset --data_files training_triplets.tsv --format raw_udd_hrt --model_name TransR --batch_size 1024 --neg_sample_size 256 --hidden_dim 200 --gamma 12.0 --lr 0.005 --max_step 10000 --log_interval 100 --batch_size_eval 1000 -adv --regularization_coef 1.00E-07 --num_thread 1 --num_proc 8 --neg_sample_size_eval 1000

In [ ]:
DGLBACKEND=pytorch dglke_train --dataset iBKH --data_path ./dataset --data_files training_triplets.tsv --format raw_udd_hrt --model_name DistMult --batch_size 1024 --neg_sample_size 256 --hidden_dim 400 --gamma 12.0 --lr 0.005 --max_step 10000 --log_interval 100 --batch_size_eval 1000 -adv --regularization_coef 1.00E-07 --num_thread 1 --num_proc 8 --neg_sample_size_eval 1000

In [ ]:
DGLBACKEND=pytorch dglke_train --dataset iBKH --data_path ./dataset --data_files training_triplets.tsv --format raw_udd_hrt --model_name ComplEx --batch_size 1024 --neg_sample_size 256 --hidden_dim 200 --gamma 12.0 --lr 0.05 --max_step 10000 --log_interval 100 --batch_size_eval 1000 -adv --regularization_coef 1.00E-07 --num_thread 1 --num_proc 8 --neg_sample_size_eval 1000